In [32]:
import cv2
import pandas as pd

# Load the CSV file
file_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\New folder\Full_length.xlsx"
data=pd.read_excel(file_path)
# Process each row in the CSV


# Load the image
image_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\right\carapace\GX010067_33_625.jpg_gamma_stabilized_00001.png" # Adjust the path as needed
image = cv2.imread(image_path)

for index, row in data.iterrows():
    # Extract the details from the row
    label = row['Label']

    bx = int(row['BX']*5.3)
    by = int(row['BY']*5.3)
    width = int(row['Width']*5.3)
    height = int(row['Height']*5.3)
    length = int(row['Length']*5.3)
    angle = row['Angle']



    print(f"Processing row {index}: {label}, bx={bx}, by={by}, width={width}, height={height}, length={length}")   

    if angle < 90  :
        start_point = (bx, by+height)
        end_point = ( bx+width,by  )
    else:
        start_point = (bx, by)
        end_point = ( bx+width,by+height  )
    color = (255, 0, 0)  # Blue color in BGR
    thickness = 2
    image = cv2.line(image, start_point, end_point, color, thickness)

    # Draw a large circle at (bx, by)
    circle_radius = 10
    circle_color = (0, 0, 255)  # Red color in BGR
    image = cv2.circle(image, (bx, by), circle_radius, circle_color, -1)

    # Draw circles at the corners of the bounding box
    corners = [(bx + width, by), (bx, by + height), (bx + width, by + height)]
    for corner in corners:
        image = cv2.circle(image, corner, circle_radius, circle_color, -1)

    # Add the bx, by text
    text_bx_by = f'({bx}, {by})'
    org_bx_by = (bx + 15, by + 15)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    text_color = (0, 255, 0)  # Green color in BGR
    text_thickness = 2
    image = cv2.putText(image, text_bx_by, org_bx_by, font, font_scale, text_color, text_thickness, cv2.LINE_AA)

    # Add the length text
    text_length = f'Length: {length:.2f}'
    org_length = (bx+width, by - 10)
    image = cv2.putText(image, text_length, org_length, font, font_scale, text_color, text_thickness, cv2.LINE_AA)


    # Save or display the image
output_path = f'annot.jpg'  # Adjust the path as needed
cv2.imwrite(output_path, image)
print(f"Saved output image to {output_path}")






Processing row 0: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=2875, by=1822, width=51, height=17, length=53
Processing row 1: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=2875, by=1822, width=51, height=17, length=284
Processing row 2: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=2878, by=524, width=489, height=462, length=672
Processing row 3: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=3643, by=1762, width=344, height=383, length=513
Processing row 4: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=997, by=1944, width=821, height=225, length=850
Processing row 5: full length:GX010067_33_625.jpg_gamma_stabilized_00001, bx=82, by=2094, width=505, height=44, length=506
Saved output image to annot.jpg


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
# from measurements_calculator import convert_pixel_to_real_length
# from src.measurement.enclosing_circle_calculation import minimum_enclosing_circle
import csv

class ImageViewer:
    """
    A class for viewing the segmentation annotated images and ImageJ results.

    Args:
        image_dir (str): The directory path where the images are stored.
        label_dir (str): The directory path where the labels are stored.
        imagej_results_file (str): The file path of the ImageJ results CSV.

    Attributes:
        image_dir (str): The directory path where the images are stored.
        label_dir (str): The directory path where the labels are stored.
        images (list): A list of image file names in the image directory.
        current_image_name (str): The file name of the current image being viewed.
        imagej_results (DataFrame): The ImageJ results loaded from the CSV file.

    Methods:
        _load_images: Load the image file names from the image directory.
        _load_labels: Load the label file for a given image.
        resize_image_to_screen: Resize an image to fit the screen dimensions.
        show_image: Show the current image with annotations.
        _draw_annotations: Draw annotations on the image.
        _draw_single_annotation: Draw a single annotation on the image.
        _convert_to_pixels: Convert normalized points to pixel coordinates.
        navigate_images: Navigate to the next or previous image.
        save_data_to_csv: Save data to a CSV file.
        run_viewer: Run the image viewer loop.
    """
    
    def __init__(self, image_dir, label_dir, imagej_results_file):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.images = self._load_images()
        self.current_image_name = self.images[0] if self.images else None
        self.imagej_results = self._load_imagej_results(imagej_results_file)
        self.labels = []

    def _load_images(self):
        """
        Load and return a list of images from the specified image directory.

        Returns:
            A list of image filenames, sorted in ascending order.
        """
        return [img for img in sorted(os.listdir(self.image_dir)) if img.endswith(('.png', '.jpg', '.jpeg'))]

    def _load_imagej_results(self, imagej_results_file):
        """
        Load the ImageJ results from the CSV file.

        Args:
            imagej_results_file (str): The path to the ImageJ results CSV file.

        Returns:
            DataFrame: The loaded ImageJ results.
        """
        return pd.read_csv(imagej_results_file)

    def _load_labels(self, image_name):
        """
        Load the labels for the given image.

        Args:
            image_name (str): The name of the image.

        Returns:
            list: A list of keypoints for each label in the image.
        """
        label_name = os.path.splitext(image_name)[0] + '.txt'
        label_path = os.path.join(self.label_dir, label_name)
        if os.path.exists(label_path):
            with open(label_path, 'r') as file:
                lines = file.readlines()
                labels = []
                for line in lines:
                    parts = line.split()
                    keypoints = [(float(parts[i]), float(parts[i + 1])) for i in range(5, len(parts), 2)]
                    labels.append(keypoints)
                return labels
        return []

    def resize_image_to_screen(self, image, screen_width, screen_height, keep_aspect_ratio=True):
        """
        Resize the given image to fit within the specified screen dimensions.

        Args:
            image (numpy.ndarray): The image to be resized.
            screen_width (int): The width of the screen.
            screen_height (int): The height of the screen.
            keep_aspect_ratio (bool, optional): Whether to maintain the aspect ratio of the image. 
                Defaults to True.

        Returns:
            numpy.ndarray: The resized image.
        """
        img_height, img_width = image.shape[:2]

        if keep_aspect_ratio:
            # Calculate the ratio of the screen dimensions to the image dimensions
            width_ratio = screen_width / img_width
            height_ratio = screen_height / img_height

            # Use the smaller ratio to ensure the entire image fits within the screen
            resize_ratio = min(width_ratio, height_ratio)
            new_width = int(img_width * resize_ratio)
            new_height = int(img_height * resize_ratio)

            image = cv2.resize(image, (new_width, new_height))
        else:
            image = cv2.resize(image, (screen_width, screen_height))

        return image

    def show_image(self):
        """
        Displays the current image in a window.

        This method loads the image specified by `self.current_image_name` from the image directory,
        resizes it to fit the screen, draws any annotations on the image, and displays it in a window.

        Returns:
            None
        """
        if self.current_image_name:
            image_path = os.path.join(self.image_dir, self.current_image_name)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Failed to load image: {image_path}")
                return
            screen_width, screen_height = 5312, 2988
            resized_image = self.resize_image_to_screen(image, screen_width, screen_height, keep_aspect_ratio=False)
            self._draw_annotations(resized_image)
            cv2.imshow('Image', resized_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

    def _draw_annotations(self, image):
        """
        Draws annotations on the given image based on the loaded labels.

        Args:
            image (numpy.ndarray): The image on which the annotations will be drawn.

        Returns:
            None
        """
        label_path = self._load_labels(self.current_image_name)
        for points in label_path:
            self._draw_single_annotation(image, points)

        imagej_result = self.imagej_results[self.imagej_results['Label'].str.contains(self.current_image_name)].iloc[0]
        bx, by = int(imagej_result['BX']), int(imagej_result['BY'])
        width, height = int(imagej_result['Width']), int(imagej_result['Height'])
        length = imagej_result['Length']
        diagonal_start = (bx, by)
        diagonal_end = (bx + width, by + height)
        cv2.line(image, diagonal_start, diagonal_end, (255, 0, 0), 2)
        cv2.putText(image, f'Length: {length:.2f}', (bx, by - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

    def _draw_single_annotation(self, image, points):
        """
        Draws a single annotation on the given image.

        Args:
            image (numpy.ndarray): The image on which to draw the annotation.
            points (list): The list of points representing the annotation.

        Returns:
            None
        """
        if len(points) > 2:
            pixel_points = self._convert_to_pixels(points, image.shape[1], image.shape[0])
            cv2.polylines(image, [np.array(pixel_points, dtype=np.int32)], isClosed=True, color=(0, 255, 0), thickness=2)
            center, radius = minimum_enclosing_circle(pixel_points)
            if center and radius:
                cv2.circle(image, (int(center[0]), int(center[1])), int(radius), (0, 0, 255), 2)
                self.length_by_calc = round(radius*2, 6)

            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            font_color = (255, 255, 255)  # White color
            line_type = 2
            # Position for the text - Adjust as needed
            text_pos = (int(center[0] + radius), int(center[1]))
            # Put the diameter text on the image
            cv2.putText(image, 
                        f"Diameter: {self.length_by_calc}", 
                        text_pos, 
                        font, 
                        font_scale, 
                        font_color, 
                        line_type)
            print(f"Length by calc: {self.length_by_calc}")
            text_pos2 = (int(center[0] ), int(center[1]+30))
            # Put the diameter text on the image
            cv2.putText(image, 
                        f"Diameter_real: {convert_pixel_to_real_length(self.length_by_calc)}", 
                        text_pos2, 
                        font, 
                        font_scale, 
                        font_color, 
                        line_type)
            print(f'diameter real: {convert_pixel_to_real_length(self.length_by_calc)}')

    def _convert_to_pixels(self, normalized_points, image_width, image_height):
        """
        Converts a list of normalized points to pixel coordinates based on the given image dimensions.

        Args:
            normalized_points (list): A list of tuples representing normalized points in the range [0, 1].
            image_width (int): The width of the image in pixels.
            image_height (int): The height of the image in pixels.

        Returns:
            list: A list of tuples representing the pixel coordinates of the normalized points.
        """
        pixel_points = []
        for x_normalized, y_normalized in normalized_points:
            x
